In [1]:
import os
import pandas as pd
import collections
import compress_pickle
import copy
import fnmatch
import itertools
import libsbml
import numpy as np
import time
import matplotlib.pyplot as plt

# Import my local AMAS package
import sys
sys.path.insert(0, '/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2')
from AMAS import recommender
from AMAS import constants as cn 
from AMAS import tools
from AMAS import species_annotation as sa

In [8]:
with open(os.path.join(cn.REF_DIR, 'chebi_low_synonyms_comp.lzma'), 'rb') as f:
  CHEBI_LOW_SYNONYMS = compress_pickle.load(f)
CHARCOUNT_COMB_DF = compress_pickle.load(os.path.join(cn.REF_DIR, 'charcount_df_scaled.lzma'),
                                         compression="lzma")
CHARCOUNT_DF = CHARCOUNT_COMB_DF.iloc[:, :-2]
CHEBI_DF = CHARCOUNT_COMB_DF.iloc[:, -2:]

In [10]:
CHARCOUNT_COMB_DF

,a,b,c,d,e,f,g,h,i,j,...,2,3,4,5,6,7,8,9,synonym,chebi
0,0.000000,0.000000,0.000000,0.000000,0.522233,0.000000,0.0,0.174078,0.348155,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,l-norepinephrine,CHEBI:18357
1,0.200000,0.000000,0.000000,0.000000,0.400000,0.200000,0.0,0.000000,0.400000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,norepinefrina,CHEBI:18357
2,0.000000,0.000000,0.000000,0.000000,0.530330,0.000000,0.0,0.176777,0.353553,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,norepinephrine,CHEBI:18357
3,0.400000,0.000000,0.000000,0.200000,0.400000,0.000000,0.0,0.000000,0.200000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,noradrenaline,CHEBI:18357
4,0.365148,0.000000,0.000000,0.182574,0.365148,0.000000,0.0,0.000000,0.182574,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,(r)-noradrenaline,CHEBI:18357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431019,0.375558,0.037556,0.112667,0.075112,0.338002,0.000000,0.0,0.262891,0.112667,0.0,...,0.375558,0.075112,0.112667,0.075112,0.037556,0.000000,0.037556,0.000000,"2-[(2s,4as,12as)-5-methyl-6-oxo-8-[(1-oxo-2-ph...",CHEBI:99995
431020,0.341593,0.136637,0.034159,0.170797,0.307434,0.068319,0.0,0.239115,0.136637,0.0,...,0.102478,0.170797,0.136637,0.068319,0.034159,0.000000,0.000000,0.068319,"n-[(1s,3s,4ar,9as)-3-[2-[(2,5-difluorophenyl)m...",CHEBI:99996
431021,0.338241,0.075165,0.150329,0.075165,0.300658,0.037582,0.0,0.225494,0.112747,0.0,...,0.338241,0.112747,0.112747,0.075165,0.037582,0.000000,0.037582,0.000000,"n-[(2s,4as,12as)-2-[2-(cyclohexylmethylamino)-...",CHEBI:99997
431022,0.257396,0.032174,0.128698,0.193047,0.289570,0.032174,0.0,0.225221,0.257396,0.0,...,0.225221,0.096523,0.064349,0.032174,0.032174,0.032174,0.032174,0.064349,"n-[[(3s,9s,10r)-16-(dimethylamino)-12-[(2s)-1-...",CHEBI:99998


In [16]:
CHARCOUNT_COMB_DF[CHARCOUNT_COMB_DF['chebi'] == 'CHEBI:16595']

,a,b,c,d,e,f,g,h,i,j,...,2,3,4,5,6,7,8,9,synonym,chebi
22018,0.124035,0.0,0.0,0.124035,0.124035,0.0,0.000000,0.248069,0.372104,0.0,...,0.0,0.000000,0.124035,0.124035,0.0,0.0,0.0,0.0,"1d-myo-inositol 1,4,5-trisphosphate",CHEBI:16595
22019,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.577350,0.0,...,0.0,0.577350,0.000000,0.000000,0.0,0.0,0.0,0.0,ip3,CHEBI:16595
22020,0.096674,0.0,0.0,0.290021,0.193347,0.0,0.096674,0.290021,0.386695,0.0,...,0.0,0.000000,0.096674,0.096674,0.0,0.0,0.0,0.0,"1d-myo-inositol 1,4,5-tris(dihydrogen phosphate)",CHEBI:16595
22021,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.353553,0.0,...,0.0,0.353553,0.353553,0.353553,0.0,0.0,0.0,0.0,"1,4,5-insp3",CHEBI:16595
22022,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.353553,0.0,...,0.0,0.353553,0.353553,0.353553,0.0,0.0,0.0,0.0,"ins(1,4,5)p3",CHEBI:16595
22023,0.132453,0.0,0.0,0.132453,0.132453,0.0,0.000000,0.264906,0.397360,0.0,...,0.0,0.000000,0.132453,0.132453,0.0,0.0,0.0,0.0,"d-myo-inositol-1,4,5-triphosphate",CHEBI:16595
22024,0.138675,0.0,0.0,0.000000,0.138675,0.0,0.000000,0.277350,0.416025,0.0,...,0.0,0.000000,0.138675,0.138675,0.0,0.0,0.0,0.0,"inositol 1,4,5-trisphosphate",CHEBI:16595
22025,0.127000,0.0,0.0,0.127000,0.127000,0.0,0.000000,0.254000,0.381000,0.0,...,0.0,0.000000,0.127000,0.127000,0.0,0.0,0.0,0.0,"d-myo-inositol 1,4,5-trisphosphate",CHEBI:16595
22026,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.447214,0.0,...,0.0,0.447214,0.000000,0.000000,0.0,0.0,0.0,0.0,insp3,CHEBI:16595


In [9]:
CHARCOUNT_DF

,a,b,c,d,e,f,g,h,i,j,...,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.000000,0.000000,0.000000,0.522233,0.000000,0.0,0.174078,0.348155,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.200000,0.000000,0.000000,0.000000,0.400000,0.200000,0.0,0.000000,0.400000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.530330,0.000000,0.0,0.176777,0.353553,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.400000,0.000000,0.000000,0.200000,0.400000,0.000000,0.0,0.000000,0.200000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.365148,0.000000,0.000000,0.182574,0.365148,0.000000,0.0,0.000000,0.182574,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431019,0.375558,0.037556,0.112667,0.075112,0.338002,0.000000,0.0,0.262891,0.112667,0.0,...,0.000000,0.187779,0.375558,0.075112,0.112667,0.075112,0.037556,0.000000,0.037556,0.000000
431020,0.341593,0.136637,0.034159,0.170797,0.307434,0.068319,0.0,0.239115,0.136637,0.0,...,0.000000,0.136637,0.102478,0.170797,0.136637,0.068319,0.034159,0.000000,0.000000,0.068319
431021,0.338241,0.075165,0.150329,0.075165,0.300658,0.037582,0.0,0.225494,0.112747,0.0,...,0.000000,0.150329,0.338241,0.112747,0.112747,0.075165,0.037582,0.000000,0.037582,0.000000
431022,0.257396,0.032174,0.128698,0.193047,0.289570,0.032174,0.0,0.225221,0.257396,0.0,...,0.096523,0.386094,0.225221,0.096523,0.064349,0.032174,0.032174,0.032174,0.032174,0.064349


In [3]:
with open(os.path.join(cn.REF_DIR, 'chebi2label.lzma'), 'rb') as f:
  REF_CHEBI2LABEL = compress_pickle.load(f)
REF_CHEBI2LABEL

{'CHEBI:18357': '(R)-noradrenaline',
 'CHEBI:10': '(+)-Atherospermoline',
 'CHEBI:133004': 'bisbenzylisoquinoline alkaloid',
 'CHEBI:100': '(-)-medicarpin',
 'CHEBI:16114': 'medicarpin',
 'CHEBI:76924': 'plant metabolite',
 'CHEBI:6714': '(+)-medicarpin',
 'CHEBI:16713': '2-aminopteridine-4,7-diol',
 'CHEBI:10000': 'Vismione D',
 'CHEBI:46955': 'anthracenes',
 'CHEBI:100000': '(2S,3S,4R)-3-[4-(3-cyclopentylprop-1-ynyl)phenyl]-4-(hydroxymethyl)-1-(2-methoxy-1-oxoethyl)-2-azetidinecarbonitrile',
 'CHEBI:22712': 'benzenes',
 'CHEBI:36820': 'ring assembly',
 'CHEBI:38777': 'azetidines',
 'CHEBI:100001': 'N-[(2R,3S,6R)-2-(hydroxymethyl)-6-[2-[[oxo-[4-(trifluoromethyl)anilino]methyl]amino]ethyl]-3-oxanyl]-3-pyridinecarboxamide',
 'CHEBI:20857': 'C-glycosyl compound',
 'CHEBI:100002': '3-chloro-N-[(5S,6S,9S)-5-methoxy-3,6,9-trimethyl-2-oxo-11-oxa-3,8-diazabicyclo[10.4.0]hexadeca-1(12),13,15-trien-14-yl]benzenesulfonamide',
 'CHEBI:24995': 'lactam',
 'CHEBI:52898': 'azamacrocycle',
 'CHEBI:100